Title

Summary

Introduction:
provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project
identify and describe the dataset that was used to answer the question


Methods & Results:
describe in written english the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.





note: all tables and figure should have a figure/table number and a legend


Discussion:
summarize what you found
discuss whether this is what you expected to find?
discuss what impact could such findings have?
discuss what future questions could this lead to?


References:
at least 4 citations relevant to the project (format is your choose, just be consistent across the references).

In [2]:
import numpy as np
import altair as alt
import pandas as pd


# your report should include code which:
# loads data from the original source on the web -> this is allowed to be downloaded

In [3]:
import os
import pandas as pd

laptops = pd.read_csv("DATA/laptops.csv")
laptops.head()
    


,index,brand,Model,Price,Rating,processor_brand,processor_tier,num_cores,num_threads,ram_memory,...,secondary_storage_type,secondary_storage_capacity,gpu_brand,gpu_type,is_touch_screen,display_size,resolution_width,resolution_height,OS,year_of_warranty
0,1,tecno,Tecno Megabook T1 Laptop (11th Gen Core i3/ 8G...,23990,63,intel,core i3,2,4,8,...,No secondary storage,0,intel,integrated,False,15.6,1920,1080,windows,1
1,2,tecno,Tecno Megabook T1 Laptop (11th Gen Core i7/ 16...,35990,67,intel,core i7,4,8,16,...,No secondary storage,0,intel,integrated,False,15.6,1920,1080,windows,1
2,3,hp,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,51100,73,amd,ryzen 5,6,12,8,...,No secondary storage,0,amd,dedicated,False,15.6,1920,1080,windows,1
3,4,acer,Acer Extensa EX214-53 Laptop (12th Gen Core i5...,39990,62,intel,core i5,12,16,8,...,No secondary storage,0,intel,integrated,False,14.0,1920,1080,windows,1
4,5,lenovo,Lenovo V15 82KDA01BIH Laptop (AMD Ryzen 3 5300...,28580,62,amd,ryzen 3,4,8,8,...,No secondary storage,0,amd,integrated,False,15.6,1920,1080,windows,1


# wrangles and cleans the data from it’s original (downloaded) format to the format necessary for the planned classification or clustering analysis

In [4]:
from sklearn.linear_model import LinearRegression
new_laptops = laptops.drop(['index', 'Model', 'secondary_storage_type', 'secondary_storage_capacity', 'OS', 'year_of_warranty', 'is_touch_screen'], axis=1)
new_laptops

# Reassign qualitative variables to dummies
new_laptops = pd.get_dummies(new_laptops)


# performs a summary of the data set that is relevant for exploratory data analysis related to the planned classification analysis


# creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned classification analysis


# performs classification or regression analysis




In [14]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split
import pandas as pd
import altair as alt




lap_train, lap_test = train_test_split(new_laptops, test_size=0.3)



X_train = lap_train.drop('Price', axis=1)
y_train = lap_train['Price']

X_test = lap_test.drop('Price', axis=1)
y_test = lap_test['Price']


steps = [
    ('scaler', StandardScaler()),  
    ('knn', KNeighborsRegressor())  
]


knn_pipeline = Pipeline(steps)


knn_fit = knn_pipeline.fit(X_train, y_train)

laptop_preprocessor = make_column_transformer((StandardScaler(), ["Price"]))
laptop_pipeline = make_pipeline(laptop_preprocessor, KNeighborsRegressor())



lap_test["Predicted"] = knn_fit.predict(X_test)

lap_test_metrics = lap_test[["Predicted", "Price"]]

cv_50_df = pd.DataFrame(
    cross_validate(
        estimator=knn_pipeline,
        cv=50,
        X=X_train,
        y=y_train
    )
)
cv_50_metrics = cv_50_df.agg(["mean", "sem"])
cv_50_metrics

parameter_grid = {
    "kneighborsregressor__n_neighbors": range(1, 100, 5),
}




# Tune

from sklearn.model_selection import GridSearchCV

sacr_gridsearch = GridSearchCV(
    estimator=knn_pipeline,
    param_grid=parameter_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
)


sacr_gridsearch.fit(
    X_train,  # A single-column data frame
    y_train  # A series
)







ValueError: Invalid parameter 'kneighborsregressor' for estimator Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsRegressor())]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [6]:
lap_test_metrics 

accuracy = knn_fit.score(X_test, y_test)
print("Accuracy:", accuracy)




            




Accuracy: 0.8408850472259136


# creates a visualization of the result of the analysis